In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [3]:
Donnée = pd.read_excel('data/ONLINE_RETAILS.xlsx')

In [4]:
Donnée

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [ ]:
Donnée.info()

In [ ]:
Donnée['InvoiceDate'] = pd.to_datetime(Donnée['InvoiceDate'])
Donnée = Donnée.set_index('InvoiceDate')

## PREPROCESSING

In [ ]:
Donnée.info()

In [ ]:
Donnée.describe()

In [ ]:
Donnée.loc['2011', 'Quantity'].resample('W').mean().plot()
plt.show()

In [ ]:
Donnée.dtypes

-------------------------------------------------------------------------------------------------

## DATA CLEANING

In [ ]:
data = Donnée.copy()

- Présence des valeurs négatives 
- Focus que sur les valeurs positives

In [ ]:
valeur_neg = data[data['Quantity']<0].head()
valeur_neg

- Transformation des données négatives en données positives

In [ ]:
data['Quantity'] = data['Quantity'].abs()

In [ ]:
data.head()

Suppression des données dupliquées

In [ ]:
data.drop_duplicates()

## MODIFICATION DU FORMAT DATE
On va utiliser InvoiceDate  

In [ ]:
data = data.reset_index()
data['Annee'] = data['InvoiceDate'].dt.year
data['Mois'] = data['InvoiceDate'].dt.month
data['Jour'] = data['InvoiceDate'].dt.day
data.dropna(inplace=True)
data = data[~(data.CustomerID.isnull())]

In [ ]:
data.head()

Creation d'une colonne Dépense = quantité * prix unitaire

In [ ]:
data['depense'] = data.Quantity * data.UnitPrice

In [ ]:
data.tail(4)

## Question 1 : VOLUME

Process : 

Données utilisées 
- Country ==> Pays où l’achat a été réalisé
- Quantity ===> Quantité de produits 
- CustomerID ==> Identifiant de l’acheteur

In [ ]:
volume = data.groupby(['CustomerID']).agg({'Quantity':'sum'})

In [ ]:
volume = volume.reset_index()
volume.head()

In [ ]:
# En général un client achète :
general_quantite = volume.Quantity.mean().__round__(2)
general_quantite
print("En général un client achète " + str(volume.Quantity.mean().__round__(2)) + " Produits")

#### Tracez la distribution associée.


In [ ]:
plt.figure(figsize=(15,8))
sns.histplot(data=volume, x="Quantity", kde=True, bins=250)

Le volume moyen et median par pays

le nombre moyen

In [ ]:
nombre_moyen= data.groupby(['Country']).agg({'Quantity':'mean'}).reset_index()

le nombre median

In [ ]:
nombre_median= data.groupby(['Country']).agg({'Quantity':'median'}).reset_index()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Question  2: MONTANT

Process : 

Données utilisées 
- Quantity : Quantité de produits achetés
- UnitPrice : Prix unitaire du produit
- CustomerID ==> Identifiant de l’acheteur

In [ ]:
data.head()

In [ ]:
montant = data.groupby(['CustomerID']).agg({'StockCode':'count'}).reset_index()
montant

In [ ]:
montant.StockCode.round(2)

In [ ]:
print("En moyenne un client achete : ", np.mean(montant['StockCode']), "produits")

### Tracez la distribution associée.

In [ ]:
plt.figure(figsize=(15,8))
sns.histplot(data=montant, x="StockCode", kde=True, bins=250)

Le Montant moyen et median par pays
- Travail à faire :
    - Calculer le montant moyen et le montant médian d’un panier client, par pays

Montant moyen par client

In [ ]:
montant_moyen = data.groupby(['Country']).agg({'depense':'mean'})

In [ ]:
montant_moyen.rewet_index()

In [ ]:
montant_moyen.head().round(2)

Montant median par client

In [ ]:
montant_median = data.groupby(['Country']).agg({'depense':'median'})


In [ ]:
montant_median.rewet_index()

In [ ]:
montant_median.head().round(2)

## Question 3 : Volume: Top-5 produits


## Premier trimestre 2011
- StockCode : identifiant de produit
- Quantity : quantité de produits achetés
- InvoiceDate : date de l’achat

In [ ]:
data_T = data_T.copy()

In [ ]:
data_T.head(5)

Top-5 produits les plus vendus 
-     1er Trimestre 2011

In [ ]:
Mask1 = (data_T['Annee']==2011)
Mask1 = data_T[Mask1]

In [ ]:
Mask1.head(3)

In [ ]:
Mask2 = (Mask1['Mois'].between(1,3))

In [ ]:
data_T = Mask1[Mask2]

In [ ]:
data_T.tail()

In [ ]:
Top_5_1 = data_T.groupby(['StockCode','Month']).agg({'Quantity':'sum'}).reset_index()

In [ ]:
Top_5_1.sort_values(['Quantity'], ascending=False).head(5)

Top-5 produits les plus vendus 
-     2em trimestre 2011

In [ ]:
Mask3 = (data_T['Annee']==2011)
Mask3 = data_T[Mask3]

In [ ]:
Mask3.head(3)

In [ ]:
Mask4 = (Mask3['Mois'].between(4,6))

In [ ]:
data_T = Mask3[Mask4]

In [ ]:
Top_5_2 = data_T.groupby(['StockCode','Month']).agg({'Quantity':'sum'}).reset_index()

In [ ]:
Top_5_2.sort_values(['Quantity'], ascending=False).head(5)

## Question 4 : Chiffre d’Affaire

In [ ]:
data_T['depense'].groupby(data_T['Country']).agg('sum')

In [ ]:
data_T.sort_values(ascending = False).head(5)

## Question 5 : Evolution Mensuelle 

-     Evolution mensuelle du Montant Moyen d’un panier client pour chacun de ces cinq pays

In [ ]:
data_T['depense'].groupby(data_T['Country']).agg('mean').sort_values(ascending = False).head(5).plot(figsize = (15, 7))


-     Evolution mensuelle du Montant Médian d’un panier client pour chacun de ces cinq pays

In [ ]:
data_T['Depense'].groupby(data_T['Country']).agg('sum').sort_values(ascending = False).head(5).plot(figsize = (15, 7))

## Question 6 : Fréquence

-     fréquence des 100 plus gros clients du Royaume.Uni

In [ ]:
data_frequency = data.copy()

In [ ]:
dt_ref = data_frequency.InvoiceDate.max()
dt_ref = dt_ref + datetime.timedelta(days = 1)

In [ ]:
dt_ref

In [ ]:
data_frequency['jours_depuis_le_dernier_achat'] = dt_ref - data_frequency.InvoiceDate

In [ ]:
data_frequency.head(5)

In [ ]:
client_hist = data_frequency.groupby("CustomerID").min().reset_index()[['CustomerID', 'num_jours_depuis_le_dernier_achat']]
client_hist.rename(columns={'num_jours_depuis_le_dernier_achat':'Achat_recent'}, inplace=True)
client_hist.Achat_recent.describe()

In [ ]:
client_hist.head()

In [ ]:
valeur = data_frequency[['CustomerID', 'Depense']].groupby("CustomerID").sum().reset_index()
client_his = client_hist.merge(valeur, how='outer')
client_hist.Depense = client_hist.depense+0.001
client_frequency = data_frequency[['CustomerID', 'depense']].groupby("CustomerID").count().reset_index()
client_frequency.rename(columns={'depense':'frequency'},inplace=True)
client_hist = client_hist.merge(client_frequency, how='outer')

In [ ]:
client_hist

Fréquence. A quelle fréquence dépensent les 100 plus gros clients du Royaume-Uni ?

In [ ]:
client_hist['Pays'] = data_frequency.Country

In [ ]:
client_hist.head()

In [ ]:
united_kingdom = client_hist[client_hist['Pays'] == 'United Kingdom'].sort_values(by=['depense'], ascending=False)
united_kingdom

In [ ]:
united_kingdom = united_kingdom.head(100)
united_kingdom

In [ ]:
frequence_dep = united_kingdom.frequency.mean()
frequence_dep

In [ ]:
display("Les 100 plus gros clients du Royaume-Uni dépensent a une fréquence de {} % ".format(frequence_dep))

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## II. Modèles

-     1.Quel sera le chiffre d’affaire global de la société en Décembre 2011 ?

In [ ]:
Msk = (data['Annee']==2011) #& (data_clean['Mois'].between(1,3))
Msk = data[Msk]

In [ ]:
Mask2 = (Msk[Msk['Mois'] == 12 ])

In [ ]:
Mask2.head(3).reset_index()

In [ ]:
Mask2.groupby(['Country']).agg({'Depense':'median'})

In [ ]:
united_kingdom_2 = Mask2[Mask2['Country'] == 'United Kingdom']
united_kingdom_2.head(2)

In [ ]:
united_kingdom_2.Country.unique()

In [ ]:
Chiffre_Affaire = united_kingdom_2.Depense.sum()

In [ ]:
print("le chiffre d’affaire global de la société en Décembre 2011 est de {} euro".format(Chiffre_Affaire))

-     2. Combien de clients vont réaliser au moins un achat en Décembre 2011 en France ?
